# Clustering

For this lab, we are going to do **unsupervised** learning on **unstructured** data!

More specificially, we will be doing **clustering** on news articles.

Along the way, we will also learn a simple way to **summarize** texts with a few representative words.

It is unsupervised learning because we do not provide the algorithms with any labeled data. There are no correct answers, just raw, unlabeled texts. We want the algorithm to find patterns in the data and divide the different texts into groups, *clusters*, such that articles that are in the same cluster are similar in some aspect. It is up to us to interpret what aspect that might be.

It is unstructured data, because each news article is just a long string of words. There are no numbers in a neat columnar structure and the articles are of different lengths.

## About the data

The data consists of 10 000 Swedish news articles, crawled from an online news site, and are a sample from news articles published during two years.

Our hope is that we will recognize both general (robberies, movie reviews) and specific (the Svenska Akademien affair) clusters of articles.

## Loading the data

Since we are dealing with unstructured data, chances are that we will *not* load it from a csv file, since these are all about structure.

This time, the data is in **JSON** format, the same format that many document (or NoSQL) databases use.
The format has its origin from JavaScript data types, but is also very close to Python's data types, in particular we will recognize dicts {} and lists [].

Actually, I lied. It is not really in JSON format, but a variant of it that I think is somewhat underused.

There is a big problem with JSON and somewhat large datasets. Imagine you have a file containing a list of ten million items. Even if you were only interested in the ten first items, the JSON decoder would still need to read all ten million items into memory, just to correctly parse the list. That is kind of harsh on the memory usage.

Fortunately, there is a simple solution.

In the **JSONL** format, each line is individually encoded as a JSON object. You can then read as few lines as you wish from the file, and don't load anything more than necessary into memory. If you *do* want to use all objects in the file, you can do so in a streaming manner, and possibly use little memory.

We will need to import `requests`, a library that makes downloading things easy, and `json` to parse the file

In [1]:
import json
import requests

In [2]:
URL = 'https://storage.googleapis.com/slides.tenfifty.io/arts_sample.jsonl'

Download the given `URL`, but keep the data in memory. No need to save to disk.

In [1]:
# ...

Now, as Python sees it, this is just one really long string.
We need to parse it. Use the `json` module to decode this string into a Python list of strings (the individual articles).

Let the variable `texts` refer to this decoded list.

Do you get the error `JSONDecodeError: Extra data: line 2 column 1`?
Remember that this is JSON**L**, not plain JSON. We will need to split the long string into individual lines and parse each line as JSON individually. A *list comprehension* would be super handy for this, but you could also append line by line to a list. 

In [2]:
# texts = 

How many articles are there? There should be 10 000.

Print the first one. You should always take a quick look at the data to see what you are dealing with

In [3]:
# print()
# print()

Good, now we are ready to go!

The problem is, we now have a bunch of strings, but typically ML algorithms work with numbers. We need a way to convert the texts.

Also, since our two tasks are to find a way to summarize the articles, and to cluster them by contents (topics), it would be nice if we could weight the words, so that common words that appear in almost all articles nostly will be ignored.

Fortunately, the `sklearn` library has our back.

It contains a class TfidfVectorizer that does exactly this.

And *this* is quite a lot! More specifically, it:

1. Splits each text up into individual words, ignoring punctuation, etc.
1. Makes a dictionary that maps words to unique numerical IDs
1. Throws away words that occur less that some limit, to get rid of spelling errors and uncommon words
1. Trows away words that are too common and thus are not helpful
1. Calculates a weight for each word in each document, such that a high weight means that the word is important. These are words that occur frequently in the current document, but has a low frequency overall in all documents.
1. Converts each document to a row in a matrix, where each column represents a specific word, and the value is the count of the word in this document * the calculated weight of the word
1. Actually makes this into a sparse matrix, since it would be huge otherwise

Not bad at all!

We start by importing the TfidfVectorizer and creating an instance of it with some suitable settings.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.3, min_df=2)

Now, we must **fit** (let the Vectorizer gather its word count statistics that it needs to compute the word weights) and **transform** (actually do this transformation on our texts) the vectorizer.

This can be done in one step. Check out the documentation.

Save the result from this operation as `X`, since it is our transformed training data set.


In [4]:
# X = 

X is now a matrix. What dimensions do you expect it to have?
You should know the number of rows, and perhaps you have a rough estimate for the number of columns?
Print the dimensions, or the *shape* of the matrix to see if you were right.

In [5]:
#

Remember that we wanted a way to summarize a document with a few keywords?

Well, the vectorizer makes this quite easy.

It has already the weights for all words, so if we just *transform* a document, it will replace the words in the document with their weights multiplied by their counts -- in short, a way of assigning importance to words. So if we just can find the highest numbers after this transformation, map them back to their word values, and sort this list, we might have something useful!

To start, transform the first text using our vectorizer

In [6]:
# result = 

Then, we need to find the score and actual words

In [10]:
feature_names = vectorizer.get_feature_names()
weights_sk = [(feature_names[col], result[0, col]) for col in result.nonzero()[1]]

Now, weights_sk is a list of tuples, where each tuple is of the form (word, score).

Can you sort this list so that the words with the highest scores come first, and print the first 10 words?

In [7]:
# 

Print the article again for comparison

In [8]:
# print()

Not a bad summary!

Finally, lets see if we can find some clusters of articles that share the same topic.

We will use a clustering algorithm called `KMeans` (not to be confused with KNN (k-Nearest Neighbors), which is a supervised learning algorithm used for both classification and regression.

In [13]:
from sklearn.cluster import KMeans

For this algorithm, we will have to specify how many clusters we expect, and also for how many iterations it should work. There are other clustering algorithms that can figure this out on their own.

In [14]:
num_clusters = 100
km = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100, n_init=1)

As you might have observed, you call `fit` when you want an algorithm in `sklearn` to learn your data.

Make the clusterer learn our training data set!

In [9]:
# 

Finally, we iterate through each cluster and print some representative words!

In [ ]:
print("Top terms per cluster:")

order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % feature_names[ind], end='')
    print()

Does the topics make sense to you?

Pretty impressive, huh, given that the algorithm does not know anything about what the words actually mean!

This is an example of how we still can make sense of data, even if it is unstructured (text) and we have no labels (unsupervised learning).

We could make this even better by giving the preprocessing of the texts some more love. We could for example transform each word into its base form (eleverna -> elev, mamman -> mamma) and remove common words that still make it through the weighting process.

We could also try a hierarchical clustering, or perform a dimensionality reduction and plot the clusters and articles in a two dimensionsal plot, with similar articles closer to each other.

Nevertheless, we reaches some pretty good results with not too many lines of code.

I hope this has been inpiring!